In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
def custom_save(function=plt.savefig, fname=""):
    if not os.path.exists(os.path.dirname(fname)):
        os.makedirs(os.path.dirname(fname))
        print('created new directory', os.path.dirname(fname))
    try:
        # to make sure suptitle is not cut off
        function(fname, bbox_inches="tight")
    except:
        function(fname)
    print("saved as", fname)

def plot_spectrogram(spectrogram, label):
    plt.figure()
    plt.pcolormesh(np.log(spectrogram))
    plt.xlabel('time index [-]')
    plt.ylabel('frequency index [-]')
    plt.title(label)
    
def plot_soundlevel(df, label, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    ax.semilogy(df.timestamp_s, df.sound_level, label=label)
    ax.set_xlabel('time [s]')
    ax.set_ylabel('sound level')

In [ ]:
from evaluate_data import  CSV_DIRNAME, WAV_DIRNAME, FS 
from evaluate_data import read_df, get_spectrogram, add_soundlevel

## Compare sound levels

In [ ]:
duration = 10 * 1e3 # miliseconds from end
degree = 0
snr = False
props = False

df_source = read_df(degree=degree, props=False, snr=snr, motors=False, source=True)
df_all = read_df(degree=degree, props=props, snr=snr, motors=True, source=True)
df_props = read_df(degree=0, props=False, snr=False, motors=True, source=False)

spectrogram_source = get_spectrogram(df_source)
spectrogram_all = get_spectrogram(df_all)
spectrogram_props = get_spectrogram(df_props)

plot_spectrogram(spectrogram_source, label="source")
plot_spectrogram(spectrogram_all, label="all")
plot_spectrogram(spectrogram_props, label="props")

add_soundlevel(df_source, duration=duration)
add_soundlevel(df_all, duration=duration)
add_soundlevel(df_props, duration=duration)

fig, ax = plt.subplots()
plot_soundlevel(df_source, "source", ax=ax)
plot_soundlevel(df_all, "all", ax=ax)
plot_soundlevel(df_props, "props", ax=ax)
plt.legend()
plt.title('audio deck signals, selected 32 frequencies')

In [ ]:
def read_df_from_wav(fname):
    import wavio
    from scipy.signal import stft
    w = wavio.read(fname)
    source_data = w.data
    
    n_buffer = 1024

    f, t, source_stft = stft(source_data, w.rate, nperseg=n_buffer, axis=0)
    source_stft = source_stft.transpose([1, 0, 2]) # channels x frequencies x times
    source_freq = np.fft.rfftfreq(n=n_buffer, d=1/w.rate)

    df = pd.DataFrame(columns=df_source.columns)
    for i in range(source_stft.shape[2]):
        df.loc[len(df), :] = {
            "index": i,
            "timestamp": i * 1024 /w.rate * 1000, # miliseconds
            "n_mics": 2,
            "topic": "measurement_mic",
            "signals_f": source_stft[:, :, i], 
            "frequencies": source_freq,
            "n_frequencies": len(source_freq)
        }
    return df

fname_source = f'{WAV_DIRNAME}/crazyflie_audio_measurements_nomotors_nosnr_noprops_source_45.wav'
fname_all =    f'{WAV_DIRNAME}/crazyflie_audio_measurements_motors_nosnr_noprops_source_45.wav'
fname_props =  f'{WAV_DIRNAME}/crazyflie_audio_measurements_motors_nosnr_noprops_nosource_45.wav'

#df_wav_all = read_df_from_wav(fname_all) 
#df_wav_source = read_df_from_wav(fname_source) 
#df_wav_props = read_df_from_wav(fname_props) 
#df_wav_all.to_pickle(fname_all.replace('.wav', '.pk'))
#df_wav_source.to_pickle(fname_source.replace('.wav', '.pk'))
#df_wav_props.to_pickle(fname_props.replace('.wav', '.pk'))

df_wav_all = pd.read_pickle(fname_all.replace('.wav', '.pk'))
df_wav_source = pd.read_pickle(fname_source.replace('.wav', '.pk'))
df_wav_props = pd.read_pickle(fname_props.replace('.wav', '.pk'))

thresh = 1e2
add_soundlevel(df_wav_source, threshold=thresh)
add_soundlevel(df_wav_all, threshold=thresh)
add_soundlevel(df_wav_props, threshold=thresh)

fig, ax = plt.subplots()
plot_soundlevel(df_wav_source, label="source", ax=ax)
plot_soundlevel(df_wav_all, label="all", ax=ax)
plot_soundlevel(df_wav_props, label="props", ax=ax)
plt.legend()
plt.title('measurement mic signals, all 512 frequencies')

In [ ]:
def select_bins(row):
    signals_f = row.signals_f[:, bins_wav_selected]
    frequencies = row.frequencies[bins_wav_selected]
    row.frequencies = frequencies
    row.n_frequencies = len(frequencies)
    return row

freqs_selected = df_source.loc[0, 'frequencies']
freqs_wav = df_wav_source.loc[0, 'frequencies']
bins_wav_selected = [np.argmin(np.abs(freq - freqs_wav)) for freq in freqs_selected]

df_wav_selected_all = df_wav_all.apply(select_bins, axis=1)
df_wav_selected_source = df_wav_source.apply(select_bins, axis=1)
df_wav_selected_props = df_wav_props.apply(select_bins, axis=1)

print(f"reduced from {len(df_wav_all.loc[0, 'frequencies'])} to {len(df_wav_selected_all.loc[0, 'frequencies'])}")

thresh = 1e2
add_soundlevel(df_wav_selected_source, duration=duration, threshold=thresh)
add_soundlevel(df_wav_selected_all, duration=duration, threshold=thresh)
add_soundlevel(df_wav_selected_props, duration=duration, threshold=thresh)

fig, ax = plt.subplots()
plot_soundlevel(df_wav_selected_source, label="source", ax=ax)
plot_soundlevel(df_wav_selected_all, label="all", ax=ax)
plot_soundlevel(df_wav_selected_props, label="props", ax=ax)
plt.legend()
plt.title('measurement mic signals, selected 32 frequencies')

## Beamforming

In [ ]:
from evaluate_data import evaluate_data

fname = 'results/static_spectra.pkl'
result_df = evaluate_data(fname)

In [ ]:
plt.figure()
plt.pcolormesh(df.timestamp_s, angles, np.log(das_spectrum))
plt.axhline(gt_degrees, color='orange')
plt.xlabel('time [s]')
plt.ylabel('angle [deg]')
plt.title(f'DAS {combine}')

plt.figure()
plt.pcolormesh(df.timestamp_s, angles, np.log(mvdr_spectrum))
plt.axhline(gt_degrees, color='orange')
plt.xlabel('time [s]')
plt.ylabel('angle [deg]')
plt.title(f'MVDR {combine}')

In [ ]:
plt.figure()
plt.pcolormesh(angles, freqs, np.log(das_spectrum_all))
plt.axvline(gt_degrees, color='orange')
plt.title('DAS')
plt.xlabel('angle [deg]')
plt.ylabel('frequency [Hz]')

plt.figure()
plt.pcolormesh(angles, freqs, np.log(mvdr_spectrum_all))
plt.axvline(gt_degrees, color='orange')
plt.title('MVDR')
plt.xlabel('angle [deg]')
plt.ylabel('frequency [Hz]')

## Plot results

In [ ]:
fname = 'results/static_spectra.pkl'
result_df = pd.read_pickle(fname)
angles = np.linspace(0, 360, 181)

## A. Ideal case: what is best combination of method+combine+normalize? 

In [ ]:
import itertools
from evaluate_data import degree_list, method_list, combine_list, normalize_list

base_dict = dict(
    motors=False,
    props=False,
    snr=False, 
    degree=None, # will be set later
    method=None,
    combine=None,
    normalize=None
)

for degree, method, normalize, combine in itertools.product(degree_list, method_list, normalize_list, combine_list):
    gt_degree = 180 - degree
    
    this_dict = base_dict.copy()
    this_dict["degree"] = degree
    this_dict["method"] = method
    this_dict["combine"] = combine
    this_dict["normalize"] = normalize
    
    this_df = result_df.copy()
    for key, val in this_dict.items():
        if not len(this_df.loc[this_df[key]==val]):
            print(f'did not find any {key}=={val} in results. Available: {this_df[key].unique()}')
        this_df = this_df.loc[this_df[key]==val]
        if not len(this_df):
            break
            
    if not len(this_df):
        continue 
        
        
    # angles x times
    full_spectrum = np.array([*this_df.loc[:, "spectrum"]]).T
    
    fig = plt.figure()
    fig.set_size_inches(10, 5)
    plt.pcolormesh(range(full_spectrum.shape[1]), angles, np.log(full_spectrum))
    plt.colorbar()
    plt.axhline(gt_degree, color='orange', ls=':')
    plt.xlabel('time idx [-]')
    plt.ylabel('angle [deg]')
    plt.title(f"{degree} deg, method:{method}, normalize:{normalize}, combine:{combine}")

## B. no motors: does it help to use high-SNR bins? 

In [ ]:
base_dict = dict(
    motors=False,
    props=False,
    snr=None, # will be set later
    degrees=None, # will be set later
    method='mvdr',
    combine='product'
)

this_dict = base_dict.copy()
this_dict["degree"] = degree
this_dict["snr"] = snr

## C. motors, no filtering: does it help to use high-SNR bins? 